<a href="https://colab.research.google.com/github/phenssen/getting-started-with-html/blob/master/Agent_Testing_Ground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from crewai import Agent, Task
from crewai_tools import (SerperDevTool)

# from tools.vector_search import VectorSearchTools

from langchain_openai import ChatOpenAI

from textwrap import dedent


# external_vector_tools = VectorSearchTools()
search_tool = SerperDevTool()

from crewai import Crew, Process

from typing import Union, List, Tuple, Dict
import json
import os
from crewai_tools import tool
from crewai_tools import BaseTool

from pydantic import BaseModel, Field
from typing import Type
from google.colab import userdata
import os

# Config

In [ ]:
standard_model = "chatgpt-4o-latest"
serper_api_key = userdata.get('SERPER_API_KEY')
os.environ['SERPER_API_KEY'] = serper_api_key
openai_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_key

# Tools

## Human Input

In [ ]:
class HumanInputSchema(BaseModel):
    prompt: str = Field(description="The prompt to show to the user")

class TerminalInputTool(BaseTool):
    name: str = "TerminalInputTool"
    description: str = "Get input from the user via terminal prompt"
    args_schema: Type[BaseModel] = HumanInputSchema

    def _run(self, prompt: str) -> str:
        """
        Get input from user via terminal.

        Args:
            prompt (str): The prompt to show to the user

        Returns:
            str: The user's input response
        """
        try:
            user_input = input(f"{prompt}\n> ")
            return user_input.strip()
        except Exception as e:
            return f"Error getting user input: {str(e)}"

# Create terminal input tool instance
human_input = TerminalInputTool()


## Calculator

In [ ]:
class CalculatorSchema(BaseModel):
    operation: str = Field(
        description="A mathematical expression to evaluate, like '200*7' or '5000/2*10'. Supports basic arithmetic operations: addition (+), subtraction (-), multiplication (*), and division (/)"
    )


class CalculatorTool(BaseTool):
    name: str = "Calculator"
    description: str = (
        """Useful to perform basic mathematical calculations, like addition, subtraction,
        multiplication, and division. The input to this tool should be a mathematical expression, a couple examples are `200*7` or `5000/2*10`"""
    )
    args_schema: Type[BaseModel] = CalculatorSchema

    def _run(self, operation: str) -> float:
        """Useful to perform basic mathematical calculations,
        like addition, subtraction, multiplication, and division.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        import ast
        import operator

        # Define allowed operators
        allowed_operators = {
            ast.Add: operator.add,
            ast.Sub: operator.sub,
            ast.Mult: operator.mul,
            ast.Div: operator.truediv,
            ast.USub: operator.neg,
        }

        def safe_eval(node):
            if isinstance(node, ast.Num):
                return node.n
            elif isinstance(node, ast.BinOp):
                if type(node.op) not in allowed_operators:
                    raise ValueError(f"Unsupported operation: {type(node.op).__name__}")
                return allowed_operators[type(node.op)](
                    safe_eval(node.left), safe_eval(node.right)
                )
            elif isinstance(node, ast.UnaryOp):
                if type(node.op) not in allowed_operators:
                    raise ValueError(f"Unsupported operation: {type(node.op).__name__}")
                return allowed_operators[type(node.op)](safe_eval(node.operand))
            else:
                raise ValueError(f"Unsupported node type: {type(node).__name__}")

        try:
            parsed = ast.parse(operation, mode="eval")
            result = safe_eval(parsed.body)
            return float(result)
        except (ValueError, TypeError, ZeroDivisionError) as e:
            raise ValueError(f"Invalid expression: {str(e)}")


NameError: name 'BaseModel' is not defined

# Agents

In [ ]:
researcher = Agent(
    role="Research Analyst",
    goal="""
        Conduct research and analysis on topics by gathering information from reliable sources.
        Synthesize findings into clear insights.
        Focus on collecting factual data and evidence to support conclusions.
        """
    ,
    backstory="""
        You are an experienced research analyst with expertise in gathering and analyzing information
        from multiple sources. Your analytical skills allow you to quickly identify relevant data points
        and synthesize them into meaningful insights. You have a strong attention to detail and ability
        to verify information through cross-referencing.
        """
    ,
    verbose=True,
    tools=[search_tool],
    llm=ChatOpenAI(model=standard_model),
    #max_iter = 5,
)

interviewer = Agent(
    role="Professional Interviewer",
    goal="""
        Conduct insightful interviews with industry experts and stakeholders to gather valuable
        perspectives on future trends, challenges, and opportunities. Ask probing follow-up
        questions to uncover deeper insights and ensure comprehensive understanding of complex topics.
        """
    ,
    backstory="""
        You are a seasoned professional interviewer with decades of experience conducting high-level
        executive interviews and expert consultations. Your interviewing style combines active listening
        with strategic questioning to draw out meaningful insights. You're known for creating a comfortable
        environment that encourages candid discussion while maintaining professional focus. Your ability
        to adapt questioning strategies and follow up on key points has made you highly respected in
        gathering expert perspectives on complex industry topics.
        """
    ,
    verbose=True,
    tools=[human_input],
    llm=ChatOpenAI(model=standard_model),
    max_iter=5,
)

new_agent = Agent(
    role="Example 1",
    goal="""
        Conduct insightful interviews with industry experts and stakeholders to gather valuable
        perspectives on future trends, challenges, and opportunities. Ask probing follow-up
        questions to uncover deeper insights and ensure comprehensive understanding of complex topics.
        """
    ,
    backstory="""
        You are a seasoned professional interviewer with decades of experience conducting high-level
        executive interviews and expert consultations. Your interviewing style combines active listening
        with strategic questioning to draw out meaningful insights. You're known for creating a comfortable
        environment that encourages candid discussion while maintaining professional focus. Your ability
        to adapt questioning strategies and follow up on key points has made you highly respected in
        gathering expert perspectives on complex industry topics.
        """
    ,
    verbose=True,
    tools=[human_input],
    llm=ChatOpenAI(model=standard_model),
    max_iter=5,
)

# Tasks

In [ ]:
question = "How is the weather in Berlin during winter?"

research_task = Task(
    description=f"""
        Conduct research to gather comprehensive information about {question}.
        """
    ,
    agent=researcher,
    expected_output="""
        A detailed research report.
        """
    ,
)

interview_task = Task(
    description=f"""
        Pick two to three concrete aspects from the context. Conduct a brief interview with the user to confirm the findings from the research about {question}.
        """
    ,
    agent=interviewer,
    expected_output="""
        A brief summary combining the findings from the research and the interview in markdown format.
        """
    ,
    context=[research_task],
)


# Crew

In [ ]:
crew = Crew(
    agents=[researcher, interviewer], # hier ggf. weitere agents wie oben definiert ergänzen
    tasks=[research_task, interview_task], # hier ggf. weitere tasks wie oben definiert ergänzen
    process=Process.sequential,
    verbose=True,
    output_log_file="interview.log",
)

crew.kickoff()